#医疗行业数据编织

##Section 1 实体识别与信息提取

问题：电子病历中广泛存在以自由文本记录的信息，如病程记录、查房记录、医患沟通记录等。这些文本是极具价值的医疗行业数据，对于跟踪患者病情演化、评估疗效、医疗循证等方面都具有重要的作用。然而，自由文本的作者和读者通常是人而不是机器或程序，并不能直接被其它程序有效利用；对于医疗工作者而言，从大量的文本中进行统计分析，总结变化趋势，如体温变化趋势、临床观察值变化趋势等，也是工作量巨大且容易出错的部分。

本实验拟采用大模型技术，通过提示工程，从自由文本中识别医疗行业概念实体，并将自由文本转换为FHIR R4版本数据资源，以便为基于这些文本数据进行统计分析、即席查询提供技术基础。

###实验大模型

本实验采用的大模型为智谱开放平台提供的[GLM-4](https://open.bigmodel.cn/)，运行之前请至该平台注册账号并生成API密钥，将密钥在本页作为全局变量输入，后续的代码中将自动凭此密钥访问GLM-4，如产生费用，将由实验者自理。

###测试数据来源

本文中采用的自由文本数据来源于[中文医疗信息处理评测基准CBLUE](https://tianchi.aliyun.com/dataset/95414)，主要采用了其中临床发现事件抽取（CHIP-CDEE）任务所用的数据。

###Samples

####Sample1

4+月前患者无明显诱因出现阴道不规则点滴流血，无畏寒、发热，无腹痛、腹胀及阴道流液，无咳嗽、咳痰等不适，当时未引起重视，为求进一步治疗，遂到某医院就诊，行宫腔镜及诊断性刮宫提示子宫内膜癌，经金域检验中心会诊后，病检结果（123456）提示：子宫腔腺癌，结合免疫表型及he形态符合低分化子宫内膜样腺癌，免疫组化结果：ck(2+)、vim(2+)、er(小灶+)、pr（-)、p16(2+)、p53(2+)、a-inhibin（-)、ki-67(2+,约80%)。

####Sample2

术后分别予以tp方案行了五周期化疗，化疗过程顺利，复查肝肾功、血常规、电解质无异常，出院后无发热、腹胀痛、阴道异常流血流液等双下肢疼痛等，有恶心、呕吐等反应，现患者要求入院择期行第六周期化疗，门诊以“1.左侧卵巢卵巢腺癌iv期术后；2.tp第六次化疗”收入院。

####Sample3

请外科会诊后建议至上级医院手术或者放化疗，但患者拒绝并办理出院；此后患者仍诉张口困难，偶有左侧面颈部轻微疼痛，无发热，无咳嗽咯痰，无腹痛腹泻，22天前患者在家属陪同下再次到我科住院治疗，入院后完善检查无化疗禁忌，征得患者及家属同意后于5-25日行紫杉醇240mg d1+奈达铂120mg d1化疗，并于5月29日给予帕米磷酸二钠抗溶骨治疗，治疗过程顺利，化疗后患者面部包块缩小，今日患者为行下一疗程化疗入院，门诊以“口腔粘膜恶性肿瘤 口腔鳞状细胞癌 t4n2M0”收治入院。

In [9]:
#运行环境设置
#在本级目录下建立文件apikey，并将密钥内容拷贝在该文件中
with open('../apikey', 'r') as file:
    api_key = file.read()
##提示词 
prompt_text = """
你是一个医疗行业数据提取助手。你能从临床文本中识别医疗信息实体，如诊断、体征、药嘱、手术、实验室检验、影像学发现等。
不要提供任何解释，也不要返回任何无关文字，只返回回答变量。
这是我希望在响应中看到的内容格式：
{
    "诊断":[实体1，实体2，...],
    "体征":[实体1，实体2，...],
    "药嘱":[{"药品名称":"","单次用量":"","用药频率":""},{"药品名称":"","单次用量":"","用药频率":""},...],
    "实验室检验":[实体1，实体2，...],
    "手术":[{"手术名称":"","手术部位":""},{"手术名称":"","手术部位":""},...]},
    "影像学发现":[{"检查方法":"","检查部位":"","检查发现":[实体1，实体2，...]},{"检查方法":"","检查部位":"","检查发现":[实体1，实体2，...]},...],
    "阴性发现":[实体1，实体2，...]
}
在"诊断"实体中只记录明确下达的诊断，不要记录实验室检查或影像学发现中的诊断。
不要在"体征"中记录阴性体征，阴性体征都记录在"阴性发现中"。
对于未出现在文中的实体，则不要在JSON中列出，例如文中没有记录手术，则不列出手术实体。不要写成"手术":["无"]"的形式。
各实体中只记录有肯定语义的发现，有否定意味的发现，都记录在"阴性发现"中，记录发现名称，去掉否定修饰词，例如"无发热，无腹痛"应记录为""阴性发现":["发热","腹痛"]"。
对于药嘱的描述，药品名称和用法用量间加一个空格，例如"格列美脲4mg、 瑞格列奈4mg qd"应写成""药嘱":["格列美脲 4mg","瑞格列奈 4mg qd"]"。
在JSON中不要打印任何前导空格、leading space("\n  ")以避免出现IndentationError: unexpected indent这样的错误。
在JSON中不要打印换行符、空格或任何其它转义字符如"\n"等。
"""

In [3]:
#请输入想要处理的自由文本，也可从本文的Samples中选填
free_text = input("请输入医疗文本: ")

In [11]:
import json,re
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key) # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": free_text},
    ],
    temperature=0.01,
    max_tokens=2048
)
#print(response.choices[0].message)
payload=response.choices[0].message.content
json_block_pattern=r"'''json\n?|'''"
jsonString=re.sub(json_block_pattern,'',payload,flags=re.DOTALL)
#print(jsonString)
try:
    data = json.loads(jsonString)
    print(data)
except json.JSONDecodeError as e:
    print("无法解析JSON字符串:",e)


{'诊断': ['左侧卵巢卵巢腺癌iv期术后'], '体征': ['恶心', '呕吐'], '药嘱': [], '实验室检验': ['肝肾功', '血常规', '电解质'], '手术': [{'手术名称': '化疗', '手术部位': '左侧卵巢'}], '影像学发现': [], '阴性发现': ['发热', '腹胀痛', '阴道异常流血流液', '双下肢疼痛']}
